In [4]:
import os
import requests
import json
import xml.etree.ElementTree as ET
import pandas as pd
import pyarrow as pa
import pyarrow.parquet as pq
import time
from datetime import datetime

from dotenv import load_dotenv, set_key

#Har ni skapat eget konto på trafikverket kan ni lägga till egen API_KEY här, annars kan ni använda min.
API_KEY = '77d239ecbcb14551b0217e6b3aa552e3'

URL = 'https://api.trafikinfo.trafikverket.se/v2/data.json'

HEADERS = {
      'Content-Type': 'text/xml'
    }

ModuleNotFoundError: No module named 'requests'

In [ ]:
### FUNKTIONER ###

def get_all_station_codes():
  request_body = f"""
  <REQUEST>
    <LOGIN authenticationkey="{API_KEY}" />
    <QUERY orderby="LocationSignature" objecttype="TrainStation" namespace="rail.infrastructure" schemaversion="1.5" limit="7000">
      <FILTER>
      </FILTER>
      <INCLUDE>AdvertisedLocationName</INCLUDE>
      <INCLUDE>LocationSignature</INCLUDE>
    </QUERY>
  </REQUEST>
  """
  try:
    print("Fetching all station data...")
    response = requests.post(
      URL, 
      data=request_body.encode('utf-8'), 
      headers=HEADERS
    )
    response.raise_for_status() # Raise an exception for bad status codes

    data = response.json()
        
      # 1. Access the list of station dictionaries
    stations = data['RESPONSE']['RESULT'][0]['TrainStation']
        
      # 2. Use a list comprehension to extract only the LocationSignature from each dictionary
    station_codes = [station['LocationSignature'] for station in stations]
        
    print(f"Successfully retrieved {len(station_codes)} station codes.")
    return station_codes

  except requests.exceptions.RequestException as e:
    print(f"An error occurred during the API call: {e}")
    return []
  except KeyError as e:
    print(f"Error: Could not find expected data field: {e}")
    print("This usually means the API structure changed or the request failed silently.")
    return []
  
def fetch_train_data(station_code, start_date_str, end_date_str):
    """
    Fetches train announcements for a single station and saves the results 
    to a Parquet file.
    
    Args:
        station_code (str): The LocationSignature of the station (e.g., 'M', 'Cst').
        start_date_str (str): Start date/time in 'YYYY-MM-DDTHH:mm:ss' format.
        end_date_str (str): End date/time in 'YYYY-MM-DDTHH:mm:ss' format.
    """
    
    # 1. Construct the XML Payload
    request_body = f"""
    <REQUEST>
      <LOGIN authenticationkey="{API_KEY}" />
      <QUERY orderby="AdvertisedTimeAtLocation desc" objecttype="TrainAnnouncement" schemaversion="1.9" limit="100000"> 
        <FILTER>
          <EQ name="LocationSignature" value="Mc" />
          <EQ name="ActivityType" value="Avgang" /> 
          <GTE name="AdvertisedTimeAtLocation" value="{start_date_str}" />
          <LTE name="AdvertisedTimeAtLocation" value="{end_date_str}" />
        </FILTER>
      </QUERY>
    </REQUEST>"""
    
    print(f"  -> Fetching data for {station_code} from {start_date_str} to {end_date_str}...")

    try:
        # 2. Make the POST API Call
        response = requests.post(
            URL, 
            data=request_body.encode('utf-8'), 
            headers=HEADERS
        )
        response.raise_for_status() # Check for HTTP errors (4xx or 5xx)

        data = response.json()
        
        # Check for API error messages
        if 'Error' in data['RESPONSE']:
             print(f"  -> ERROR received for {station_code}: {data['RESPONSE']['Error']['Message']}")
             return

        announcements = data['RESPONSE']['RESULT'][0]['TrainAnnouncement']
        
        if not announcements:
            print(f"  -> No data found for {station_code} in this period. Skipping.")
            return

        print(f"  -> SUCCESS! Retrieved {len(announcements)} records.")

        # 3. Transform to Pandas DataFrame
        df = pd.DataFrame(announcements)
        
        # Add metadata columns for analysis purposes (when the data was pulled)
        df['StationCode'] = station_code
        df['RunDate'] = datetime.now().strftime('%Y-%m-%d')
        df['PullTimestamp'] = datetime.now()
        
        return df

    except requests.exceptions.RequestException as e:
        print(f"  -> API Request Failed for {station_code}: {e}")
    except KeyError:
        print(f"  -> Parsing Error: Data structure not as expected for {station_code}.")

# Om ni vill testa att öppna parquet-filen
def look_at_dataset():
    """ 
    Use to load the dataset (used when making the script in jupyter notebook)

    Returns:
        df (dataframe): dataframe with all the data. 
    """
    #Förutsatt att den ligger i samma map, annars får ni ändra path
    file_path = 'data/train_data_week_1.parquet'

    df = pd.read_parquet(file_path)
    return df

In [3]:
### "MAIN" ### 
kort_lista = ['Mc', 'A']
station_codes = get_all_station_codes()
list_of_dataframes = []

# Ändra till for code in station_codes för att köra alla 1500 stationer, använd kort_lista för testkörningar
for code in kort_lista:
    df = fetch_train_data(code, '2025-09-25T00:00:00', '2025-10-01T00:00:00')

    if df is not None:
        list_of_dataframes.append(df)

if list_of_dataframes:
    final_df = pd.concat(list_of_dataframes, ignore_index=True)

output_filename = f"train_data_week_1.parquet"
final_df.to_parquet(output_filename, index=False, compression='snappy')

Fetching all station data...
Successfully retrieved 1540 station codes.
  -> Fetching data for Mc from 2025-09-25T00:00:00 to 2025-10-01T00:00:00...
  -> SUCCESS! Retrieved 561 records.
  -> Fetching data for A from 2025-09-25T00:00:00 to 2025-10-01T00:00:00...
  -> SUCCESS! Retrieved 561 records.


In [2]:
import pandas as pd
import os

def load_and_combine_data(base_path='data'):
    """ 
    Läser in data från både arrivals- och departures-parquet-filerna 
    och returnerar en enda DataFrame. ActivityType-kolumnen antas 
    redan finnas i filerna.

    Args:
        base_path (str): Grundmappen där filerna ligger. Standard är 'data'.

    Returns:
        pd.DataFrame: En DataFrame som innehåller all data från båda filerna.
    """
    
    # 1. Definiera filerna
    files_to_load = [
        'train_data_week_2_arrivals.parquet',
        #'train_data_week_2_departures.parquet'
    ]
    
    all_data_frames = []

    # 2. Loopa igenom filerna och ladda dem
    for filename in files_to_load:
        file_path = os.path.join(base_path, filename)
        
        try:
            df = pd.read_parquet(file_path)
            all_data_frames.append(df)
            print(f"✅ Laddade in data ({len(df):,} rader) från: {file_path}")
            
        except FileNotFoundError:
            print(f"❌ Filen hittades inte: {file_path}. Hoppar över.")
            
    
    # 3. Slå samman alla DataFrames till en stor DataFrame
    if all_data_frames:
        # pd.concat staplar DataFrames på varandra
        combined_df = pd.concat(all_data_frames, ignore_index=True)
        print(f"\n✅ Sammanfogning klar! Totala rader i DataFrame: {len(combined_df):,}")
        return combined_df
    else:
        print("⚠️ Inga filer kunde laddas. Returnerar tom DataFrame.")
        return pd.DataFrame()

df_all_data = load_and_combine_data()


✅ Laddade in data (71,136 rader) från: data\train_data_week_2_arrivals.parquet

✅ Sammanfogning klar! Totala rader i DataFrame: 71,136


In [6]:
#Function for cleaning the dataframe
def clean_timestamp(df: pd.DataFrame):
    """

    """
    list_of_columns = ['AdvertisedTimeAtLocation', 'DepartureDateOTN', 'LocationDateTimeOTN', 'EstimatedTimeAtLocation', 'TimeAtLocation', 'ScheduledDepartureDateTime', 'RunDate', 'TimeAtLocationWithSeconds']
    df_cleaned = df.copy()

    for column in list_of_columns:
        df_cleaned[column] = pd.to_datetime(df_cleaned[column], errors='coerce')

        if df_cleaned[column].dt.tz is not None:
            df_cleaned[column] = df_cleaned[column].dt.tz_localize(None)
    return df_cleaned



def clean_list_dict(df: pd.DataFrame):
    list_of_columns = ['Deviation', 'OtherInformation', 'ViaFromLocation', 'OperationalTransportIdentifiers', 'ProductInformation', 'FromLocation', 'ToLocation', 'TypeOfTraffic']
    df_cleaned = df.copy()

    for column in list_of_columns:
        df_cleaned[column] = df_cleaned[column].str[0]
        new_columns = pd.json_normalize(df_cleaned[column])
        df_cleaned = pd.concat([df_cleaned, new_columns], axis = 1)
        df_cleaned.drop(columns=[column], inplace=True)
    return df_cleaned

def drop_columns(df: pd.DataFrame):

    list_of_columns = ['NewEquipment', 'WebLink', 'Service']
    df_cleaned = df.copy()
    df_cleaned.drop(list_of_columns, axis=1)

    return df_cleaned

def clean_dataframe(df: pd.DataFrame):

    df_start = df.copy()
    df_time = clean_timestamp(df_start)
    df_dict = clean_list_dict(df_time)
    df_drop = drop_columns(df_dict)
    df_cleaned = df_drop

    return df_cleaned

In [7]:
df_all_data_cleaned = clean_dataframe(df_all_data)

In [24]:
df_all_data.iloc[1]

ActivityId                                            1500adde-8c5d-45ca-08dd-f96a3c1df63b
ActivityType                                                                       Ankomst
Advertised                                                                            True
AdvertisedTimeAtLocation                                     2025-10-07T23:53:00.000+02:00
AdvertisedTrainIdent                                                                  3493
Canceled                                                                             False
Deleted                                                                              False
DepartureDateOTN                                             2025-10-07T00:00:00.000+02:00
Deviation                                [{'Code': 'ANA055', 'Description': 'Spårändrat'}]
EstimatedTimeAtLocation                                                               None
EstimatedTimeIsPreliminary                                                           False

In [22]:
from_location = df_all_data['ViaFromLocation'].iloc[8]
to_location = df_all_data['ToLocation'].iloc[1]
# Skriv ut värdet
print("Kom från:")
print(from_location)
print("Ska till:")
print(to_location)


Kom från:
[{'LocationName': 'Vå', 'Order': 0, 'Priority': 1}
 {'LocationName': 'Ör', 'Order': 1, 'Priority': 2}
 {'LocationName': 'Hpbg', 'Order': 2, 'Priority': 4}
 {'LocationName': 'Sk', 'Order': 3, 'Priority': 3}]
Ska till:
[{'LocationName': 'A', 'Order': 0, 'Priority': 1}]


In [23]:
df_all_data.head(20).ToLocation

0     [{'LocationName': 'A', 'Order': 0, 'Priority':...
1     [{'LocationName': 'A', 'Order': 0, 'Priority':...
2     [{'LocationName': 'Sk', 'Order': 0, 'Priority'...
3     [{'LocationName': 'A', 'Order': 0, 'Priority':...
4     [{'LocationName': 'G', 'Order': 0, 'Priority':...
5     [{'LocationName': 'A', 'Order': 0, 'Priority':...
6     [{'LocationName': 'Sk', 'Order': 0, 'Priority'...
7     [{'LocationName': 'A', 'Order': 0, 'Priority':...
8     [{'LocationName': 'G', 'Order': 0, 'Priority':...
9     [{'LocationName': 'A', 'Order': 0, 'Priority':...
10    [{'LocationName': 'T', 'Order': 0, 'Priority':...
11                                                 None
12    [{'LocationName': 'A', 'Order': 0, 'Priority':...
13    [{'LocationName': 'G', 'Order': 0, 'Priority':...
14    [{'LocationName': 'A', 'Order': 0, 'Priority':...
15    [{'LocationName': 'Vå', 'Order': 0, 'Priority'...
16    [{'LocationName': 'G', 'Order': 0, 'Priority':...
17    [{'LocationName': 'G', 'Order': 0, 'Priori

In [10]:
# Konfiguration för att visa alla kolumner och en bred visning samt fler rader
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1000)
pd.set_option('display.max_rows', 200)

In [3]:
#df_arrivals = look_at_dataset()
df_departures = look_at_dataset()

In [11]:
df_arrivals

,ActivityId,ActivityType,Advertised,AdvertisedTimeAtLocation,AdvertisedTrainIdent,Canceled,Deleted,DepartureDateOTN,Deviation,EstimatedTimeAtLocation,...,ViaFromLocation,WebLink,WebLinkName,OtherInformation,TrainComposition,Service,ViaToLocation,StationCode,RunDate,PullTimestamp
0,1500adde-8c5d-45ca-08dd-f579d4c8634b,Ankomst,True,2025-10-02T23:54:00.000+02:00,1152,False,False,2025-10-02T00:00:00.000+02:00,"[{'Code': 'ANA031', 'Description': 'Kort tåg'}]",2025-10-02T23:57:00.000+02:00,...,"[{'LocationName': 'Dk.cph', 'Order': 0, 'Prior...",http://www.skanetrafiken.se,Skånetrafiken,None,None,None,None,A,2025-10-03,2025-10-03 10:33:56.795397
1,1500adde-8c5d-45ca-08dd-f57a4f7f5563,Ankomst,True,2025-10-02T23:52:00.000+02:00,547,False,False,2025-10-02T00:00:00.000+02:00,None,2025-10-02T23:53:00.000+02:00,...,"[{'LocationName': 'Nr', 'Order': 0, 'Priority'...",http://www.sj.se/trafikinfo,SJ,"[{'Code': 'ONA080', 'Description': 'Ingen påst...",None,None,None,A,2025-10-03,2025-10-03 10:33:56.795397
2,1500adde-8c5d-45ca-08dd-f579dd759f4f,Ankomst,True,2025-10-02T23:52:00.000+02:00,1290,False,False,2025-10-02T00:00:00.000+02:00,None,2025-10-02T23:53:00.000+02:00,...,"[{'LocationName': 'Vid', 'Order': 0, 'Priority...",http://www.skanetrafiken.se,Skånetrafiken,None,None,None,None,A,2025-10-03,2025-10-03 10:33:56.795397
3,1500adde-8c5d-45ca-08dd-f579f41dcf12,Ankomst,True,2025-10-02T23:51:00.000+02:00,1741,False,False,2025-10-02T00:00:00.000+02:00,None,None,...,"[{'LocationName': 'Lkö', 'Order': 0, 'Priority...",http://www.skanetrafiken.se,Skånetrafiken,None,None,None,None,A,2025-10-03,2025-10-03 10:33:56.795397
4,1500adde-8c5d-45ca-08dd-f579d2654dd4,Ankomst,True,2025-10-02T23:48:00.000+02:00,11159,False,False,2025-10-02T00:00:00.000+02:00,None,None,...,"[{'LocationName': 'Mdn', 'Order': 0, 'Priority...",http://www.oresundstag.se,Öresundståg,None,None,None,None,A,2025-10-03,2025-10-03 10:33:56.795397
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1727875,1500adde-8c5d-45ca-08dd-f4afa1695242,Ankomst,True,2025-10-01T01:10:00.000+02:00,1162,False,False,2025-10-01T00:00:00.000+02:00,None,2025-10-01T01:25:00.000+02:00,...,"[{'LocationName': 'Dk.cph', 'Order': 0, 'Prior...",http://www.skanetrafiken.se,Skånetrafiken,None,None,None,None,Öäg,2025-10-03,2025-10-03 11:00:29.353503
1727876,1500adde-8c5d-45ca-08dd-f4afabb4ff3f,Ankomst,True,2025-10-01T01:06:00.000+02:00,1295,False,False,2025-10-01T00:00:00.000+02:00,None,None,...,"[{'LocationName': 'Kgå', 'Order': 0, 'Priority...",http://www.skanetrafiken.se,Skånetrafiken,None,None,None,None,Öäg,2025-10-03,2025-10-03 11:00:29.353503
1727877,1500adde-8c5d-45ca-08dd-f4afa1648d9a,Ankomst,True,2025-10-01T00:55:00.000+02:00,1160,False,False,2025-10-01T00:00:00.000+02:00,None,None,...,"[{'LocationName': 'Dk.cph', 'Order': 0, 'Prior...",http://www.skanetrafiken.se,Skånetrafiken,None,None,None,None,Öäg,2025-10-03,2025-10-03 11:00:29.353503
1727878,1500adde-8c5d-45ca-08dd-f4afb58283ca,Ankomst,True,2025-10-01T00:44:00.000+02:00,1491,False,False,2025-10-01T00:00:00.000+02:00,None,2025-10-01T00:48:00.000+02:00,...,"[{'LocationName': 'Fud', 'Order': 0, 'Priority...",http://www.skanetrafiken.se,Skånetrafiken,None,None,None,None,Öäg,2025-10-03,2025-10-03 11:00:29.353503


In [12]:
df_departures

,ActivityId,ActivityType,Advertised,AdvertisedTimeAtLocation,AdvertisedTrainIdent,Canceled,Deleted,DepartureDateOTN,Deviation,EstimatedTimeIsPreliminary,FromLocation,InformationOwner,LocationDateTimeOTN,LocationSignature,ModifiedTime,NewEquipment,Operator,OperationalTrainNumber,OperationalTransportIdentifiers,PlannedEstimatedTimeAtLocationIsValid,ProductInformation,ScheduledDepartureDateTime,TimeAtLocation,TimeAtLocationWithSeconds,ToLocation,TrackAtLocation,TrainOwner,TypeOfTraffic,ViaToLocation,WebLink,WebLinkName,MobileWebLink,OtherInformation,TrainComposition,EstimatedTimeAtLocation,Service,Booking,StationCode,RunDate,PullTimestamp
0,1500adde-8c5d-45ca-08dd-f579d4cd27f2,Avgang,True,2025-10-02T23:57:00.000+02:00,1152,False,False,2025-10-02T00:00:00.000+02:00,"[{'Code': 'ANA031', 'Description': 'Kort tåg'}]",False,"[{'LocationName': 'Dk.kh', 'Order': 0, 'Priori...",Öresundståg,2025-10-02T23:57:00.000+02:00,Mc,2025-10-02T21:59:49.578Z,0.0,TDEV,11152,"[{'Company': '3178', 'Core': '------766508', '...",False,"[{'Code': 'PNA044', 'Description': 'Öresundstå...",2025-10-02T00:00:00.000+02:00,2025-10-02T23:59:00.000+02:00,2025-10-02T23:59:34.000+02:00,"[{'LocationName': 'Lu', 'Order': 0, 'Priority'...",4b,Ö-TÅG,"[{'Code': 'YNA001', 'Description': 'Tåg'}]","[{'LocationName': 'Blv', 'Order': 0, 'Priority...",http://www.oresundstag.se,Öresundståg,None,None,None,None,None,None,A,2025-10-03,2025-10-03 09:56:22.399008
1,1500adde-8c5d-45ca-08dd-f579f41dcf13,Avgang,True,2025-10-02T23:57:00.000+02:00,1741,False,False,2025-10-02T00:00:00.000+02:00,None,False,"[{'LocationName': 'Hb', 'Order': 0, 'Priority'...",Skånetrafiken,2025-10-02T23:57:00.000+02:00,Mc,2025-10-02T21:58:37.125Z,0.0,ARRIVA,11741,"[{'Company': '0074', 'Core': '----LTP49736', '...",False,"[{'Code': 'PNA020', 'Description': 'Pågatågen'}]",2025-10-02T00:00:00.000+02:00,2025-10-02T23:58:00.000+02:00,2025-10-02T23:58:36.000+02:00,"[{'LocationName': 'Y', 'Order': 0, 'Priority':...",6,SKANE,"[{'Code': 'YNA001', 'Description': 'Tåg'}]","[{'LocationName': 'Ox', 'Order': 0, 'Priority'...",http://www.skanetrafiken.se,Skånetrafiken,http://mobil.skanetrafiken.se,"[{'Code': 'ONA147', 'Description': 'Direkt til...",None,None,None,None,A,2025-10-03,2025-10-03 09:56:22.399008
2,1500adde-8c5d-45ca-08dd-f579d2654dd5,Avgang,True,2025-10-02T23:54:00.000+02:00,11159,False,False,2025-10-02T00:00:00.000+02:00,None,False,"[{'LocationName': 'G', 'Order': 0, 'Priority':...",Öresundståg,2025-10-02T23:54:00.000+02:00,Mc,2025-10-02T21:58:42.945Z,0.0,TDEV,11159,"[{'Company': '3989', 'Core': '-------11159', '...",False,"[{'Code': 'PNA044', 'Description': 'Öresundstå...",2025-10-02T00:00:00.000+02:00,2025-10-02T23:58:00.000+02:00,2025-10-02T23:58:22.000+02:00,"[{'LocationName': 'Hie', 'Order': 0, 'Priority...",3b,Ö-TÅG,"[{'Code': 'YNA001', 'Description': 'Tåg'}]","[{'LocationName': 'Tri', 'Order': 0, 'Priority...",http://www.oresundstag.se,Öresundståg,None,None,None,None,None,None,A,2025-10-03,2025-10-03 09:56:22.399008
3,1500adde-8c5d-45ca-08dd-f579dd759f50,Avgang,True,2025-10-02T23:54:00.000+02:00,1290,False,False,2025-10-02T00:00:00.000+02:00,None,False,"[{'LocationName': 'Trg', 'Order': 0, 'Priority...",Skånetrafiken,2025-10-02T23:54:00.000+02:00,Mc,2025-10-02T21:56:24.686Z,0.0,ARRIVA,1290,"[{'Company': '3126', 'Core': '--------1290', '...",False,"[{'Code': 'PNA020', 'Description': 'Pågatågen'}]",2025-10-02T00:00:00.000+02:00,2025-10-02T23:56:00.000+02:00,2025-10-02T23:56:08.000+02:00,"[{'LocationName': 'Lu', 'Order': 0, 'Priority'...",4a,SKANE,"[{'Code': 'YNA001', 'Description': 'Tåg'}]","[{'LocationName': 'Blv', 'Order': 0, 'Priority...",http://www.skanetrafiken.se,Skånetrafiken,http://mobil.skanetrafiken.se,None,None,None,None,None,A,2025-10-03,2025-10-03 09:56:22.399008
4,1500adde-8c5d-45ca-08dd-f579e620f728,Avgang,True,2025-10-02T23:46:00.000+02:00,1440,False,False,2025-10-02T00:00:00.000+02:00,"[{'Code': 'ANA055', 'Description': 'Spårändrat'}]",False,"[{'LocationName': 'Mc', 'Order': 0, 'Priority'

In [13]:
# Sätt ditt önskade tågnummer
target_train_ident = '1152'

# Använd logisk indexering för att filtrera DataFrame
specific_train_df = df_departures[df_departures['AdvertisedTrainIdent'] == target_train_ident]

# Visa resultatet
specific_train_df

,ActivityId,ActivityType,Advertised,AdvertisedTimeAtLocation,AdvertisedTrainIdent,Canceled,Deleted,DepartureDateOTN,Deviation,EstimatedTimeIsPreliminary,FromLocation,InformationOwner,LocationDateTimeOTN,LocationSignature,ModifiedTime,NewEquipment,Operator,OperationalTrainNumber,OperationalTransportIdentifiers,PlannedEstimatedTimeAtLocationIsValid,ProductInformation,ScheduledDepartureDateTime,TimeAtLocation,TimeAtLocationWithSeconds,ToLocation,TrackAtLocation,TrainOwner,TypeOfTraffic,ViaToLocation,WebLink,WebLinkName,MobileWebLink,OtherInformation,TrainComposition,EstimatedTimeAtLocation,Service,Booking,StationCode,RunDate,PullTimestamp
0,1500adde-8c5d-45ca-08dd-f579d4cd27f2,Avgang,True,2025-10-02T23:57:00.000+02:00,1152,False,False,2025-10-02T00:00:00.000+02:00,"[{'Code': 'ANA031', 'Description': 'Kort tåg'}]",False,"[{'LocationName': 'Dk.kh', 'Order': 0, 'Priori...",Öresundståg,2025-10-02T23:57:00.000+02:00,Mc,2025-10-02T21:59:49.578Z,0.0,TDEV,11152,"[{'Company': '3178', 'Core': '------766508', '...",False,"[{'Code': 'PNA044', 'Description': 'Öresundstå...",2025-10-02T00:00:00.000+02:00,2025-10-02T23:59:00.000+02:00,2025-10-02T23:59:34.000+02:00,"[{'LocationName': 'Lu', 'Order': 0, 'Priority'...",4b,Ö-TÅG,"[{'Code': 'YNA001', 'Description': 'Tåg'}]","[{'LocationName': 'Blv', 'Order': 0, 'Priority...",http://www.oresundstag.se,Öresundståg,None,None,None,None,None,None,A,2025-10-03,2025-10-03 09:56:22.399008
569,1500adde-8c5d-45ca-08dd-f4afa117ec6b,Avgang,True,2025-10-01T23:57:00.000+02:00,1152,False,False,2025-10-01T00:00:00.000+02:00,None,False,"[{'LocationName': 'Dk.kh', 'Order': 0, 'Priori...",Öresundståg,2025-10-01T23:57:00.000+02:00,Mc,2025-10-01T21:59:11.811Z,0.0,TDEV,11152,"[{'Company': '3178', 'Core': '------766508', '...",False,"[{'Code': 'PNA044', 'Description': 'Öresundstå...",2025-10-01T00:00:00.000+02:00,2025-10-01T23:58:00.000+02:00,2025-10-01T23:58:56.000+02:00,"[{'LocationName': 'Lu', 'Order': 0, 'Priority'...",4b,Ö-TÅG,"[{'Code': 'YNA001', 'Description': 'Tåg'}]","[{'LocationName': 'Blv', 'Order': 0, 'Priority...",http://www.oresundstag.se,Öresundståg,None,None,None,None,None,None,A,2025-10-03,2025-10-03 09:56:22.399008
1131,1500adde-8c5d-45ca-08dd-f579d4cd27f2,Avgang,True,2025-10-02T23:57:00.000+02:00,1152,False,False,2025-10-02T00:00:00.000+02:00,"[{'Code': 'ANA031', 'Description': 'Kort tåg'}]",False,"[{'LocationName': 'Dk.kh', 'Order': 0, 'Priori...",Öresundståg,2025-10-02T23:57:00.000+02:00,Mc,2025-10-02T21:59:49.578Z,0.0,TDEV,11152,"[{'Company': '3178', 'Core': '------766508', '...",False,"[{'Code': 'PNA044', 'Description': 'Öresundstå...",2025-10-02T00:00:00.000+02:00,2025-10-02T23:59:00.000+02:00,2025-10-02T23:59:34.000+02:00,"[{'LocationName': 'Lu', 'Order': 0, 'Priority'...",4b,Ö-TÅG,"[{'Code': 'YNA001', 'Description': 'Tåg'}]","[{'LocationName': 'Blv', 'Order': 0, 'Priority...",http://www.oresundstag.se,Öresundståg,None,None,None,None,None,None,Aal,2025-10-03,2025-10-03 09:56:23.607952
1700,1500adde-8c5d-45ca-08dd-f4afa117ec6b,Avgang,True,2025-10-01T23:57:00.000+02:00,1152,False,False,2025-10-01T00:00:00.000+02:00,None,False,"[{'LocationName': 'Dk.kh', 'Order': 0, 'Priori...",Öresundståg,2025-10-01T23:57:00.000+02:00,Mc,2025-10-01T21:59:11.811Z,0.0,TDEV,11152,"[{'Company': '3178', 'Core': '------766508', '...",False,"[{'Code': 'PNA044', 'Description': 'Öresundstå...",2025-10-01T00:00:00.000+02:00,2025-10-01T23:58:00.000+02:00,2025-10-01T23:58:56.000+02:00,"[{'LocationName': 'Lu', 'Order': 0, 'Priority'...",4b,Ö-TÅG,"[{'Code': 'YNA001', 'Description': 'Tåg'}]","[{'LocationName': 'Blv', 'Order': 0, 'Priority...",http://www.oresundstag.se,Öresundståg,None,None,None,None,None,None,Aal,2025-10-03,2025-10-03 09:56:23.607952
2262,1500adde-8c5d-45ca-08dd-f579d4cd27f2,Avgang,True,2025-10-02T23:57:00.000+02:00,1152,False,False,2025-10-02T00:00:00.000+02:00,"[{'Code': 'ANA031', 'Description': 'Kort tåg'}]",False,"[{'LocationName': 'Dk.kh', 'Order': 0, 'Priori...",Öresundståg,2025-10-02T23:57:00.000+02:00,Mc,20